In [3]:
import numpy as np
import pandas as pd

all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/1887271186.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [4]:
all_data.columns


Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [5]:
# 注意一下，这里的列名和huili最终的sample中的列名有一点大小写区别
cv_columns = ['cv_id', 'currentPosition', 'desiredPosition', 'industry', 'desiredIndustry', 'majorName', 'skills', 'eduTracks', 'jobTracks', 'projectTracks']
jd_columns = ['jd_id', 'title', 'category_name', 'tags', 'description', 'requirement']

In [6]:
jd_data = all_data[jd_columns]
jd_data
# '''
# 可以做3个向量
# 1.title + category_name + tags
# 2.description
# 3.requirement
# 每个向量的维度在20维左右
# '''


,jd_id,title,category_name,tags,description,requirement
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...
...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...


In [8]:
# 将title + category_name + tags合并成一个字段

# step1: 缺失值填充
jd_data['title'].fillna('', inplace=True)
jd_data['category_name'].fillna('', inplace=True)
jd_data['tags'].fillna('[]', inplace=True)


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/207131757.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/207131757.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['category_name'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/207131757.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [13]:
# step2: 定义新列，将title + category_name合并
def merge_title_category_name_fun(series):
    title = series['title']
    category_name = series['category_name']
    return title + ' ' + category_name

jd_data['title_category'] = jd_data.apply(merge_title_category_name_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/1919854961.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category'] = jd_data.apply(merge_title_category_name_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理
...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java


In [14]:
# step3 将新列title_category通过jieba分词处理成列表
import re
import json
import jieba

def col_jieba_fun(series):
    col = series['title_category']

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/jieba.cache
Loading model cost 0.744 seconds.
Prefix dict has been built successfully.
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/2706112485.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]"
...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]"


In [15]:
# step4 将新列tags通过jieba分词处理成列表
import re
import json
import jieba

def col_jieba_fun(series):
    col = series['tags']

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/2841311902.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]"
...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ..."


In [17]:
# step5 将title_category_jieba和tags_jieba两个列表合并
def merge_title_category_tags_jieba(series):
    title_category_jieba = series['title_category_jieba']
    tags_jieba = series['tags_jieba']
    return title_category_jieba + tags_jieba

jd_data['title_category_tags_jieba'] = jd_data.apply(merge_title_category_tags_jieba, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/3579319595.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_tags_jieba'] = jd_data.apply(merge_title_category_tags_jieba, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划..."
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ..."
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P..."
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ..."
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风..."
...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术..."
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材..."
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,..."
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ...","[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,..."


In [18]:
#step6 过滤title_category_tags_jieba, 得到以空格分割的字符串
from zhconv import convert

def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series['title_category_tags_jieba']
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_33326/239255605.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba,title_category_tags_jieba_filter
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划...",市场营销 负责人 市场营销 营销 管理 销售 战略规划 管理工作
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ...",会员 管理 经理 用户 运营 会员 运营 会员 管理 管理 营销 社群 运营
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P...",产品 市场 经理 产品 经理 excel office ppt ui
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ...",海外 区域 运营 经理 高级 经理 运营 管理 运营 团队 管理 电商 游戏
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风...",高级 项目经理 高级 管理 项目管理 推广 归档 风险 评估 交货
...,...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术...",技术 经理 机械设备 工程师 材料 考核 维修 技术标准 不锈钢
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材...",中央 市场总监 市场总监 市场推广 医疗器械 语言 材料 营销
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,...",中央 市场总监 市场 营销 市场推广 医疗器械 语言 材料 营销
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ...","[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,...",java 开发 工程师 方向 java java 服务 性能 优化 sql 架构 需求 分析


In [19]:
jd_data[['title_category_tags_jieba', 'title_category_tags_jieba_filter']]

,title_category_tags_jieba,title_category_tags_jieba_filter
0,"[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划...",市场营销 负责人 市场营销 营销 管理 销售 战略规划 管理工作
1,"[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ...",会员 管理 经理 用户 运营 会员 运营 会员 管理 管理 营销 社群 运营
2,"[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P...",产品 市场 经理 产品 经理 excel office ppt ui
3,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ...",海外 区域 运营 经理 高级 经理 运营 管理 运营 团队 管理 电商 游戏
4,"[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风...",高级 项目经理 高级 管理 项目管理 推广 归档 风险 评估 交货
...,...,...
81291,"[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术...",技术 经理 机械设备 工程师 材料 考核 维修 技术标准 不锈钢
81292,"[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材...",中央 市场总监 市场总监 市场推广 医疗器械 语言 材料 营销
81293,"[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,...",中央 市场总监 市场 营销 市场推广 医疗器械 语言 材料 营销
81294,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,...",java 开发 工程师 方向 java java 服务 性能 优化 sql 架构 需求 分析


In [26]:
#step7 将title_category_tags_jieba_filter转成tfidf向量
# 转成tfidf向量
def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'title_category_tags_jieba_filter')
tfidf


,0,1,2,3,4,5,6,7,8,9,...,3597,3598,3599,3600,3601,3602,3603,3604,3605,3606
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
vectorizer.vocabulary_

{'市场营销': 1878,
 '负责人': 3210,
 '营销': 3076,
 '管理': 2826,
 '销售': 3454,
 '战略规划': 2035,
 '管理工作': 2832,
 '会员': 1136,
 '经理': 2904,
 '用户': 2633,
 '运营': 3336,
 '社群': 2753,
 '产品': 1053,
 '市场': 1872,
 'excel': 288,
 'office': 597,
 'ppt': 665,
 'ui': 856,
 '海外': 2505,
 '区域': 1421,
 '高级': 3594,
 '团队': 1598,
 '电商': 2647,
 '游戏': 2544,
 '项目经理': 3520,
 '项目管理': 3519,
 '推广': 2143,
 '归档': 1966,
 '风险': 3550,
 '评估': 3178,
 '交货': 1049,
 '阿里': 3473,
 '智能': 2321,
 '金融': 3422,
 '行业': 3101,
 '解决方案': 3134,
 '架构师': 2384,
 '公有': 1257,
 'it': 442,
 '技术支持': 2072,
 '数据库': 2227,
 'devops': 231,
 '架构': 2383,
 '售前': 1564,
 '软件产品': 3305,
 'os': 611,
 '沟通': 2459,
 '能力': 3023,
 '软件': 3304,
 '测试': 2493,
 '反馈': 1502,
 '通信': 3366,
 '专卖店': 948,
 '店长': 1931,
 '门店': 3464,
 '调配': 3206,
 '信息': 1187,
 '收集': 2187,
 '旅行': 2281,
 '攻略': 2191,
 '内容': 1279,
 '策划': 2809,
 '调度': 3200,
 'ip': 430,
 '声学': 1676,
 '工程师': 1855,
 '技术': 2067,
 '音频': 3515,
 'ws': 907,
 '审核': 1776,
 '耳机': 2996,
 '结构设计': 2916,
 '仿真': 1121,
 'dsp': 253,
 'cocos2d': 1

In [31]:
# 小插曲 
# 'blockchainindustryresearcher': 103,
for index, value in enumerate(tfidf[103]):
    if value:
        print("{}: {}".format(index, value))

# 显示的这几条数据都是来源于同一个jd
# 但为何有不同的jd_id呢？

632: 0.7127845196278593
47272: 0.7127845196278593
76449: 0.7127845196278593


In [38]:
jd_data.iloc[47272]

jd_id                                                2c92071580b3a1010180d6de56f20740
title                                                    Blockchainindustryresearcher
category_name                                                                    后端开发
tags                                                    ["区块链","调研","调查","软件","数据分析"]
description                         1. 熟悉区块链行业未来发展的趋势，挖掘行业内有创新、潜力的项目及热点，针对项目的赛道进行深...
requirement                         1.在区块链行业至少具有1-2年的项目调研从业经验；2.信息安全、计算机、金融、数学专业研究...
title_category                                      Blockchainindustryresearcher 后端开发
title_category_jieba                      [Blockchainindustryresearcher,  , 后, 端, 开发]
tags_jieba                                      [区块, 链,  , 调研,  , 调查,  , 软件,  , 数据分析]
title_category_tags_jieba           [Blockchainindustryresearcher,  , 后, 端, 开发, 区块...
title_category_tags_jieba_filter     blockchainindustryresearcher 开发 区块 调研 调查 软件 数据分析
Name: 47272, dtype: object

In [40]:
#step8 对tfidf降维到n维
def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

,0,1,2,3,4,5,6,7,8,9
0,-0.042800,-0.158416,-0.004038,0.164229,-0.007673,0.003984,0.001506,-0.021777,-0.033931,-0.030467
1,-0.236844,0.085825,-0.230649,-0.025117,-0.002664,0.011410,0.020963,-0.038235,0.000767,-0.037497
2,-0.190874,0.194128,0.343127,0.055052,0.033979,0.041247,-0.054510,0.005465,-0.048187,0.074340
3,-0.372098,0.191220,-0.314979,-0.012905,-0.015061,0.001358,0.020440,0.001638,-0.028449,0.113055
4,0.005622,-0.057164,0.001193,-0.038629,0.003241,0.000492,-0.038812,-0.021918,-0.067039,-0.072569
...,...,...,...,...,...,...,...,...,...,...
81291,0.034486,-0.046806,0.050004,-0.045034,-0.024671,-0.013041,-0.034680,-0.015065,0.001272,-0.025947
81292,0.015164,-0.080297,0.009706,-0.010150,-0.032657,-0.000302,-0.042514,-0.022640,-0.044071,-0.033703
81293,-0.005332,-0.084554,0.012111,0.034583,-0.044229,0.004790,-0.037143,-0.020142,-0.048496,-0.017504
81294,0.547198,0.404906,-0.158577,0.200428,0.078385,0.038910,0.045109,-0.024265,0.010940,0.047499


In [16]:
jd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   jd_id                 81296 non-null  object
 1   title                 81296 non-null  object
 2   category_name         81296 non-null  object
 3   tags                  81296 non-null  object
 4   description           81284 non-null  object
 5   requirement           80758 non-null  object
 6   title_category        81296 non-null  object
 7   title_category_jieba  81296 non-null  object
 8   tags_jieba            81296 non-null  object
dtypes: object(9)
memory usage: 5.6+ MB


In [10]:
jd_data

,jd_id,title,category_name,tags,description,requirement
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...
...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...


In [60]:
cv_data = all_data[cv_columns].drop(['industry', 'desiredIndustry'], axis=1)
cv_data
'''
majorName字段是不是不太可信
eduTracks放弃

可以做如下4个向量：
1.currentPosition + desiredPosition
2.skills
3.jobTracks
4.projectTracks
'''

,cv_id,currentPosition,desiredPosition,majorName,skills,eduTracks,jobTracks,projectTracks
0,2c9207157bf91042017c0d2beef00470,副总裁/副总经理,NaN,国际经济与贸易,"[""Ipd"",""App"",""Kpi"",""Crm"",""View"",""优化"",""策划"",""营销""...","['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc..."
1,2c9207157bf91042017c0d979cbb047d,数字化营销经理,NaN,电气工程及其自动化,"[""H5"",""C1"",""Seo"",""Sem"",""Top"",""Kpi"",""Crm"",""优化"",...","['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运..."
2,2c9207157bf91042017c7232fbab3be1,NaN,NaN,Marketing,"[""Based"",""Control"",""Sap Crm"",""Improve"",""Indivi...","['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN
3,2c9207157d308592017d4a94cb1113a1,泰国国家经理,NaN,企业管理,"[""App"",""Resume"",""运营"",""策划"",""营销"",""搭建"",""笔译"",""舞蹈"",...","['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主..."
4,2c9207157dabb0a3017dbca16cdf4c26,项目经理,项目经理/主管,英语,"[""智能硬件""]","['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20..."
...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,生产工、包装工,生产制造/机械设计/制造/机械设备工程师,无,电气,"['{""endDate"": ""2022-07"", ""unified"": false, ""de...","['{""endDate"": ""2022-07"", ""companyName"": ""江门市新会...","['{""name"": null}']"
81292,8a69d7c282199453018247bac4a36859,高级市场总监,市场/市场/营销/市场营销,上海大学,"搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写","['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']"
81293,8a69d7c282199453018247bac4a36859,高级市场总监,市场/市场/营销/市场营销,上海大学,"搭建,翻译,功能,医学,数据库,Education,产品管理,产品经理,医疗器械,英语听说读写","['{""schoolTag"": ""211"", ""endDate"": ""1999-07"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""上海微创医...","['{""name"": null}']"
81294,8a69d7db826df6c201827b4338cd60c2,java,技术/后端开发/Java,太原理工大学,"Bi,Cdn,Elk,Git,Yii,App,Php,Ext,Etl,Java,Sina,H...","['{""endDate"": ""2015-06"", ""unified"": false, ""de...","['{""endDate"": ""2020-07"", ""companyName"": ""金山云"",...","['{""name"": null}']"


In [58]:
cv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cv_id            81296 non-null  object
 1   currentPosition  78952 non-null  object
 2   desiredPosition  51408 non-null  object
 3   industry         15671 non-null  object
 4   desiredIndustry  9345 non-null   object
 5   majorName        80395 non-null  object
 6   skills           74527 non-null  object
 7   eduTracks        81293 non-null  object
 8   jobTracks        80324 non-null  object
 9   projectTracks    55283 non-null  object
dtypes: object(10)
memory usage: 6.2+ MB


In [64]:
cv_data['eduTracks'][0]

'[\'{"degree": "本科", "endDate": "2008-07", "id": "2c9207157dbedf8d017dc28fbc5d336d", "majorName": "国际经济与贸易", "schoolName": "湖南科技学院", "startDate": "2004-09"}\']'

In [65]:
cv_data['jobTracks'][1]

'[\'{"companyName": "北京美餐好客科技有限公司", "companyNature": "民营", "department": "战略项目部", "description": "工作描述: 美餐成立 10 年，年 GMV 可达 20 亿。全国领先的企业级餐饮消费平台，合作企业上万家，与世界 500 强索\\\\n迪斯联手战略合作项目。\\\\n工作内容 :\\\\n1.根据公司战略规划，规划年度营销策略，带领团队策划营销活动并实施；\\\\n2.搭建营销体系，管理机制；\\\\n3.建立用户体系，包含会员体系，私域体系等，确立目标，并实施运营；\\\\n4.搭建 CRM 系统，私域小程序，线上线下闭环系统；\\\\n5.制定线上线下全消费链路服务标准，提升用户体验，满意度；\\\\n6.整合线上线下资源，带领团队与外部资源合作；\\\\n7.消费者数据分析，挖掘市场潜在用户，寻找机会点；", "endDate": "2021-09", "id": "2c9207157c9f1919017c9f1c6fab3e10", "industry": "餐饮业", "jobTitle": "数字化营销经理", "startDate": "2020-06"}\', \'{"companyName": "上海俊造进出口贸易有限公司", "companyNature": "民营", "description": "工作描述: 源自丹麦的母婴洗护类\\\\n在职期间主要业绩:\\\\n从 0-1 搭建会员体系；私域运营粉丝数量增长 3 倍。\\\\n工作内容:\\\\n1.搭建全域 CRM 体系，全年 CRM 规划制定；\\\\n2.建立顾客互动体系，策划会员活动；\\\\n3.私域运营体系搭建，基于腾讯生态与阿里生态双渠道运营。\\\\n4.招募品牌新会员，策划会员裂变；\\\\n5.淘系内容营销机构合作，图文、短视频、直播；\\\\n6.线上店铺运营，产品销售规划，活动排期报名。", "endDate": "2020-06", "id": "2c9207157c9f1919017c9f1c6fab3e11", "industry": "快速消费品(食品、饮料、化妆品)", "jobTitle": "经理", "startDate": "2019-03"}\', 

In [63]:
cv_data['projectTracks'][0]

'[\'{"companyName": "*深圳市蜂联科技有限公司（奇虎360）", "description": "项目负责人\\\\n项目描述:\\\\n经过13年、14年、15年3年的发展以及有资本方的支持，整个360智能网络设备得到了很大的发展，在一些重点渠道比如天猫、京东、苏宁、唯品会等都已经全面超越了小米，仅次于TP-LINK，整个电商在公司内部的话语权，销售占比等都在增加。\\\\n同事也暴露了很多问题:\\\\n1、 由于资本的介入，前期对利润关注的少，但是按照财务的核算，整个部门亏损严重；\\\\n2、 销售不错，传统渠道布局也完成，团队的危机意识在降低，战斗力明显感觉在下降；\\\\n3、 当时的市场，小米除了路由器，还有插线板、中继器、等一系列的产品可以和360竞争，但是我们当时只有路由器；\\\\n4、整个市场部门以前都偏向于促销卖货，做大的销售case,忽略了用户、产品本身的研究。\\\\n项目职责:\\\\n1、挑选了3位核心成员，建立了作战指挥部；\\\\n2、主导本次牵涉的渠道、市场、产品的调整；\\\\n3、要求财务、商务提供完整的成本分析以及费用标准；\\\\n4、结合竞争对手，分析了360充电器、数据线等的市场机会。\\\\n1、统一了和财务、商务成本核算的口径，明确了销售产品的成本，确保不能低于销售成本出货；\\\\n2、除传统的电商渠道外，挑选了核心销售在银行、拼多多、积分、礼品渠道下功夫，不到半年时间，这些渠道的月销最高就达到了300万；\\\\n3、重新梳理核心产品的主图，详情、评价等数据，重新找痛点，平均转化率都提升了1个点；\\\\n4、制定了至少后面3年电商操作的策略重产品、重用户，轻促销的基本原则；\\\\n5、通过对其余机会产品的研究，360充电器被公司引进，最高月达到了10万pcs；\\\\n6、以阶段渠道战役，阶段转化率提升战役，整个团队基本上996，战斗继续进行；\\\\n7、组织架构以及考核方案都得到了调整，从以前单一的销售组织，转变为市场组织，从最开始的一味销量考核，到现在的产品、利润、增长的考虑，确保了公司每年销售目标的超额完成。", "endDate": "2016-03", "id": "2c9207157dbedf8d017dc28fbc5d336a", "projectName": "组织变

In [18]:
df=pd.DataFrame({
    'a':pd.Series([np.NaN, np.nan, pd.NaT, '', None, 'I stay']),'b':pd.Series([np.NaN, 2, 3, 4, None, 6])})
df

,a,b
0,NaN,NaN
1,NaN,2.0
2,NaT,3.0
3,,4.0
4,None,NaN
5,I stay,6.0


In [19]:
# 原来np.NaN, pd.NaT和None都会被判为空值。但是''不会被判为空值。
df.isnull()

,a,b
0,True,True
1,True,False
2,True,False
3,False,False
4,True,True
5,False,False


In [20]:
# 深刻理解axis
df.isnull().sum(axis=0)

a    4
b    2
dtype: int64